#### General Initializing

In [145]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes ;
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


#### Using the BeautifulSoup lib to get the data from the wikipedia page:

In [147]:
from bs4 import BeautifulSoup

page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text;
soup = BeautifulSoup(page);

#### Finding the table within the page code:

In [148]:
table = soup.find('table', { 'class' : 'wikitable sortable' })
# print (table.prettify())

#### Parsing the table for the cell values and checking the lists returned:

In [149]:
PostalCode =  []
Borough = []
Neighbourhood = []

for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        PostalCode.append(cells[0].find(text=True))
        Borough.append( cells[1].find(text=True).replace('\n',''))
        Neighbourhood.append( cells[2].find(text=True).replace('\n',''))

In [150]:
print(PostalCode[:5])
print(Borough[:5])
print(Neighbourhood[:5])

['M1A', 'M2A', 'M3A', 'M4A', 'M5A']
['Not assigned', 'Not assigned', 'North York', 'North York', 'Downtown Toronto']
['Not assigned', 'Not assigned', 'Parkwoods', 'Victoria Village', 'Harbourfront']


#### Converting lists into a DataFrame:

In [151]:
df = pd.DataFrame();
df['PostalCode'] = PostalCode;
df['Borough'] = Borough;
df['Neighbourhood'] = Neighbourhood;
df.head(5)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Removing not assigned post codes:

In [152]:
df = df[df.Borough != 'Not assigned'].reset_index()
df.head(5)

,index,PostalCode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M5A,Downtown Toronto,Regent Park
4,6,M6A,North York,Lawrence Heights


#### Grouping all Neighbourhoods by PostalCode:

In [186]:
g1 = df.groupby(["PostalCode","Borough"], as_index=False).agg( ', '.join)
g1.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Assigning borough name to unnamed neighbourhoods:

In [194]:
g1.loc[g1['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = g1['Borough']
g1[g1['PostalCode'].str.match('M7A')]

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


#### Sanity check:

In [155]:
g1.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### DataFrame shape:

In [174]:
g1.shape

(103, 3)

#### Getting geo data from csv file:

In [209]:
coord = pd.read_csv('https://cocl.us/Geospatial_data')
coord.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
coord.head(5)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merging data from both DataFrames based on the PostalCode key:

In [223]:
g2 = pd.merge(g1, coord, on='PostalCode')
g2.head(25)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [238]:
def getNearbyVenues(names, latitudes, longitudes, radius=1200):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'QJRPO3O3JO0YXS5GMUFZW1GAFJS3DN5MVK0NDPNRPTXOIEVY', 
            'GIAY0VSCRC2YPTBBBRTAB5PDTCESSU1YWVROSEETQVV1BLYX', 
            '20180605', 
            lat, 
            lng, 
            radius, 
            200)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [250]:
toronto_venues = getNearbyVenues(names=g2['Borough'].unique(),
                                   latitudes=g2['Latitude'],
                                   longitudes=g2['Longitude']
                                  )

Scarborough
North York
East York
East Toronto
Central Toronto
Downtown Toronto
York
West Toronto
Queen's Park
Mississauga
Etobicoke


In [291]:
print(toronto_venues.shape)
toronto_venues.head(5)

(276, 3)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Scarborough,43.806686,-79.194353
1,Scarborough,43.806686,-79.194353
2,Scarborough,43.806686,-79.194353
3,Scarborough,43.806686,-79.194353
4,Scarborough,43.806686,-79.194353


In [252]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,42,42,42,42,42,42
Downtown Toronto,21,21,21,21,21,21
East Toronto,20,20,20,20,20,20
East York,25,25,25,25,25,25
Etobicoke,47,47,47,47,47,47
Mississauga,16,16,16,16,16,16
North York,4,4,4,4,4,4
Queen's Park,23,23,23,23,23,23
Scarborough,18,18,18,18,18,18


In [253]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 92 uniques categories.


In [258]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(5)

,Neighborhood,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,Bakery,Bank,Beach,Beer Store,Bookstore,...,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Yoga Studio,Zoo Exhibit
0,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Scarborough,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [259]:
toronto_onehot.shape

(276, 93)

In [264]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,Bakery,Bank,Beach,Beer Store,Bookstore,...,Thrift / Vintage Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Yoga Studio,Zoo Exhibit
0,Central Toronto,0.000000,0.000000,0.02381,0.00,0.095238,0.023810,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.02381,0.000000
1,Downtown Toronto,0.000000,0.000000,0.00000,0.00,0.047619,0.000000,0.000000,0.000000,0.047619,...,0.00,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.047619,0.00000,0.000000
2,East Toronto,0.000000,0.000000,0.00000,0.00,0.050000,0.000000,0.000000,0.000000,0.000000,...,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,East York,0.000000,0.000000,0.00000,0.04,0.000000,0.040000,0.000000,0.040000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
4,Etobicoke,0.000000,0.042553,0.00000,0.00,0.021277,0.021277,0.000000,0.021277,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.021277,0.021277,0.021277,0.00000,0.000000
5,Mississauga,0.000000,0.000000,0.00000,0.00,0.000000,0.062500,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
6,North York,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
7,Queen's Park,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.086957,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.00000,0.000000
8,Scarborough,0.055556,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.00000,0.055556
9,West Toronto,0.000000,0.000000,0.00000,0.00,0.064516,0.032258,0.000000,0.032258,0.000000,...,0.00,0.032258,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000


In [265]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
               venue  freq
0        Coffee Shop  0.12
1             Bakery  0.10
2  Indian Restaurant  0.10
3           Pharmacy  0.07
4     Sandwich Place  0.05


----Downtown Toronto----
                  venue  freq
0              Pharmacy  0.14
1  Fast Food Restaurant  0.10
2        Sandwich Place  0.10
3        Breakfast Spot  0.10
4         Train Station  0.05


----East Toronto----
               venue  freq
0  Indian Restaurant  0.10
1        Pizza Place  0.10
2        Coffee Shop  0.10
3   Department Store  0.05
4       Burger Joint  0.05


----East York----
                  venue  freq
0           Pizza Place  0.12
1  Fast Food Restaurant  0.08
2           Coffee Shop  0.08
3      Greek Restaurant  0.04
4                  Park  0.04


----Etobicoke----
                  venue  freq
0  Fast Food Restaurant  0.09
1           Coffee Shop  0.06
2     Indian Restaurant  0.06
3    Chinese Restaurant  0.06
4      Asian Restaurant  0.04


----Mississauga----


In [266]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [313]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Bakery,Indian Restaurant,Pharmacy,Wings Joint,Hotel,Sandwich Place,Hakka Restaurant,Music Store,Pizza Place
1,Downtown Toronto,Pharmacy,Sandwich Place,Fast Food Restaurant,Breakfast Spot,Grocery Store,Bowling Alley,Pizza Place,Coffee Shop,Restaurant,Rock Club
2,East Toronto,Coffee Shop,Pizza Place,Indian Restaurant,Supermarket,Pharmacy,Department Store,Clothing Store,Chinese Restaurant,Sandwich Place,Juice Bar
3,East York,Pizza Place,Coffee Shop,Fast Food Restaurant,Sports Bar,Gym / Fitness Center,Fried Chicken Joint,Food & Drink Shop,Filipino Restaurant,Discount Store,Liquor Store
4,Etobicoke,Fast Food Restaurant,Indian Restaurant,Chinese Restaurant,Coffee Shop,Furniture / Home Store,Light Rail Station,Asian Restaurant,Burger Joint,Park,Caribbean Restaurant


In [271]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [314]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 0, 2, 1, 1, 1], dtype=int32)

In [315]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [316]:
toronto_merged = toronto_venues 
toronto_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude
0,Scarborough,43.806686,-79.194353
1,Scarborough,43.806686,-79.194353
2,Scarborough,43.806686,-79.194353
3,Scarborough,43.806686,-79.194353
4,Scarborough,43.806686,-79.194353


In [317]:

# merge toronto_groued with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,43.806686,-79.194353,1,Fast Food Restaurant,Zoo Exhibit,Spa,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Cosmetics Shop,Department Store,Fruit & Vegetable Store,Garden Center
1,Scarborough,43.806686,-79.194353,1,Fast Food Restaurant,Zoo Exhibit,Spa,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Cosmetics Shop,Department Store,Fruit & Vegetable Store,Garden Center
2,Scarborough,43.806686,-79.194353,1,Fast Food Restaurant,Zoo Exhibit,Spa,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Cosmetics Shop,Department Store,Fruit & Vegetable Store,Garden Center
3,Scarborough,43.806686,-79.194353,1,Fast Food Restaurant,Zoo Exhibit,Spa,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Cosmetics Shop,Department Store,Fruit & Vegetable Store,Garden Center
4,Scarborough,43.806686,-79.194353,1,Fast Food Restaurant,Zoo Exhibit,Spa,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Cosmetics Shop,Department Store,Fruit & Vegetable Store,Garden Center


In [318]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters